In [1]:
batch_size = 5000

In [2]:
import numpy as np
from keras.datasets import mnist

%time (X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

CPU times: user 1.15 s, sys: 551 ms, total: 1.7 s
Wall time: 1.7 s
(60000, 'train samples')
(10000, 'test samples')


In [3]:
%time P = np.load('P.npy') # load pre-computed joint probabilities

CPU times: user 563 µs, sys: 1.46 s, total: 1.46 s
Wall time: 1.46 s


In [15]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import skflow

eps = 10e-8

def tsne_model(X, P):
    d = 2
    activations = skflow.ops.dnn(X, [500, 500, 2000, d], activation=tf.nn.relu) 
    v = d - 1.
    sum_act = tf.reduce_sum(tf.square(activations), reduction_indices=1)
    Q = tf.reshape(sum_act, [-1, 1]) + -2 * tf.matmul(activations, tf.transpose(activations))
    Q = (sum_act + Q) / v
    Q = tf.pow(1 + Q, -(v + 1) / 2)
    Q *= 1 - tf.identity(Q) # set the diagonal to zero..?
    Q /= tf.reduce_sum(Q)
    Q = tf.maximum(Q, eps)
    C = tf.log((P + eps) / (Q + eps))
    C = tf.reduce_sum(P * C)
    return activations, C

In [ ]:
classifier = skflow.TensorFlowEstimator(
    n_classes=0,
    model_fn=tsne_model,
    batch_size=batch_size,
    steps=1*batch_size,
    learning_rate=0.001
)
classifier.fit(X_train, P.reshape(-1, batch_size))

In [4]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.objectives import categorical_crossentropy

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 75.0% of memory, CuDNN 4007)
/usr/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [78]:
import theano
from theano.tensor.extra_ops import fill_diagonal
from keras import backend as K

# P is the joint probabilities for this batch (Keras loss functions call this y_true)
# activations is the low-dimensional output (Keras loss functions call this y_pred)
def tsne(P, activations):
    # v = length(network{end}.bias_upW) - 1
#     v = K.shape(activations)[1] - 1. # using this is much slower than setting a constant
    v = 2. - 1. # d = 2
    
    # sum_act = sum(activations .^ 2, 2) # usually matlab sums are along the first axis
    sum_act = K.sum(K.square(activations), axis=1)
    # Q = (1 + (bsxfun(@plus, sum_act, bsxfun(@plus, sum_act', -2 * activations * activations')) ./ v)) .^ -((v + 1) / 2)
    Q = K.reshape(sum_act, [-1, 1]) + -2 * K.dot(activations, K.transpose(activations))
    Q = (sum_act + Q) / v
    Q = K.pow(1 + Q, -(v + 1) / 2)
    # Q(1:n+1:end) = 0
    fill_diagonal(Q, 0) # Theano-only
    # Q = Q ./ sum(Q(:)) # sum(Q(:)) means "sum all elements in Q" http://es.mathworks.com/matlabcentral/newsreader/view_thread/51252
    Q /= K.sum(Q)
    # Q = max(Q, eps);
    Q = K.maximum(Q, K.epsilon())
    # C = sum(sum(P{1} .* log((P{1} + eps) ./ (Q + eps)))) # sum(sum(A)) also means sum all elements
    C = K.log((P + K.epsilon()) / (Q + K.epsilon()))
    C = K.sum(P * C)
    return C

In [79]:
model = Sequential()
model.add(Dense(500, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(2000))
model.add(Activation('relu'))
model.add(Dense(2))

sgd = SGD(lr=100) # even with a huge learning rate the loss doesn't seem to change?
%time model.compile(loss=tsne, optimizer=sgd)

CPU times: user 2.67 s, sys: 70.1 ms, total: 2.74 s
Wall time: 2.73 s


In [80]:
Y_train = P.reshape(X_train.shape[0], -1)
print(X_train.shape)
print(Y_train.shape)

(60000, 784)
(60000, 5000)


In [81]:
!mkdir -p plotter

In [82]:
%matplotlib inline
import gc
from matplotlib import pyplot as plt
import keras
class Plotter(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.batch = 0
        
    def on_batch_end(self, batch, logs={}):
        prediction = self.model.predict(X_test)
        fig = plt.figure(figsize=(8,8))
        plt.scatter(prediction[:,0], prediction[:,1], alpha=1, marker='o', s=3, edgecolor='', c=y_test)
        ax = fig.gca()
        ax.set_autoscale_on(False)
        fig.tight_layout()
        plt.savefig('plotter/%04d.png' % (self.batch+1), pad_inches=0)
        plt.close()
        gc.collect()
        self.batch += 1

In [83]:
model.fit(X_train, Y_train,
          batch_size=batch_size,
#           callbacks=[Plotter()],
          shuffle=False,
          nb_epoch=4,
          verbose=1)

Epoch 1/4
60000/60000 [==============================] - 7s - loss: 3.4809     
Epoch 2/4
60000/60000 [==============================] - 7s - loss: 3.4809     
Epoch 3/4
60000/60000 [==============================] - 7s - loss: 3.4809     
Epoch 4/4
60000/60000 [==============================] - 7s - loss: 3.4809     
